In [60]:
import xgboost as xgb     # extreme gradient boosting (XGB)
import pandas as pd
import shap
import geopandas as gpd
import matplotlib.pyplot as plt

In [52]:
all_data = pd.read_csv('../data/all_data_clean.csv')

In [54]:
all_data

,FIPS,POPULATION,EP_UNEMP,EP_PCI,EP_POV,EP_NOVEH,EP_NOHSDP,EP_MOBILE,EP_MINRTY,EP_CROWD,...,EP_AGE17,AFAM,WHITE,HISPANIC,WFIR_EALT,HRCN_EALT,CFLD_EALT,RFLD_EALT,HWAV_EALT,EVICTIONS
0,39061025102GID,7692.0,5.0,32194.0,6.9,4.4,4.0,0.0,8.6,1.9,...,27.9,3,348,6,0.000000,35962.23000,0.0000,3207.936,19582.0800,5.0
1,42101027000GID,NaN,11.1,24890.0,13.3,16.5,10.7,0.0,89.1,1.3,...,20.0,67,6,2,0.000000,28.75464,0.0000,0.000,32203.0900,NaN
2,06031000300GID,NaN,14.1,20563.0,10.9,0.7,2.8,0.0,51.3,0.0,...,28.9,0,0,0,1.027438,0.00000,0.0000,4501.852,30801.0700,NaN
3,24025301402GID,NaN,3.4,30319.0,13.3,0.0,12.2,0.0,34.3,1.1,...,24.7,39,127,4,0.000000,5797.42000,17529.2500,3258.663,4861.6500,NaN
4,51097950500GID,2999.0,8.0,23307.0,15.2,4.4,14.8,15.7,21.2,3.4,...,15.0,22,91,0,1336.851000,49086.93000,116263.1000,35882.730,6284.8320,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72903,34025809902GID,NaN,1.6,72165.0,7.4,3.7,4.5,0.0,11.5,0.0,...,25.4,0,189,2,7549.366000,28967.89000,714.0855,5578.049,8278.4070,NaN
72904,09009141100GID,NaN,6.2,36837.0,4.3,8.3,3.1,0.0,55.5,4.5,...,23.3,33,67,8,0.000000,266.93070,0.0000,0.000,418.3008,NaN
72905,29095009200GID,NaN,0.8,47961.0,4.8,3.7,3.6,1.0,16.5,0.0,...,19.2,3,215,3,0.000000,0.00000,0.0000,0.000,78162.4200,NaN
72906,06029005700GID,NaN,12.0,20857.0,9.6,0.7,0.8,0.0,32.0,1.0,...,42.9,0,0,0,12454.350000,0.00000,0.0000,0.000,407.3065,NaN


In [55]:
X_var = ['EP_UNEMP',
 'EP_PCI',
 'EP_NOVEH',
 'EP_MOBILE',
 'EP_CROWD',
 'EP_AGE65',
 'EP_AGE17',
 'AFAM',
 'WHITE',
 'HISPANIC',
 'WFIR_EALT',
 'HRCN_EALT',
 'CFLD_EALT',
 'RFLD_EALT',
 'HWAV_EALT',
 'POPULATION']
Y_var = ['EVICTIONS']
geospatial_columns = ['STATEFP',
                      'COUNTYFP',
                      'TRACTCE',
                      'AFFGEOID',
                      'GEOID',
                      'NAME',
                      'LSAD',
                      'ALAND',
                      'AWATER',
                      'geometry']

In [56]:
data_selected = all_data[X_var]
model = xgb.Booster()
model.load_model('../models/xgboost.model')
# drop nulls


In [62]:
preds = model.predict(xgb.DMatrix(data_selected.values))
preds

array([20.089855 , 41.836483 , 37.04977  , ..., 54.392185 , -1.3027422,
       68.49146  ], dtype=float32)

In [63]:
t_explainer = shap.TreeExplainer(model)
t_shap_values = t_explainer(data_selected)
shap_values = t_explainer.shap_values(data_selected)

In [64]:
shap_values

array([[ -1.3162799 ,  -2.046322  ,  -2.007526  , ...,  -0.3212904 ,
         -2.0157518 ,  15.557734  ],
       [ -2.2506154 ,  -5.679394  ,  13.626263  , ...,  -2.5741014 ,
         -1.709512  ,  31.44342   ],
       [  4.293972  ,   0.80132234, -25.479273  , ...,  -0.7948237 ,
          1.4860463 ,  29.99343   ],
       ...,
       [  1.6068522 ,  -5.7846503 ,  -0.56912756, ...,   0.47832412,
         -2.4284525 ,  37.341507  ],
       [  1.0263933 ,   0.66900504, -20.03817   , ...,  -0.33641934,
         -3.9579544 ,  25.791721  ],
       [ -5.449732  ,   4.9542065 ,  11.095141  , ...,   0.5448204 ,
          0.41743246,  37.280815  ]], dtype=float32)

In [65]:
shap_cols = [col + '_SHAP_VAL' for col in X_var]
shap_df = pd.DataFrame(shap_values, columns=shap_cols)
shap_df

,EP_UNEMP_SHAP_VAL,EP_PCI_SHAP_VAL,EP_NOVEH_SHAP_VAL,EP_MOBILE_SHAP_VAL,EP_CROWD_SHAP_VAL,EP_AGE65_SHAP_VAL,EP_AGE17_SHAP_VAL,AFAM_SHAP_VAL,WHITE_SHAP_VAL,HISPANIC_SHAP_VAL,WFIR_EALT_SHAP_VAL,HRCN_EALT_SHAP_VAL,CFLD_EALT_SHAP_VAL,RFLD_EALT_SHAP_VAL,HWAV_EALT_SHAP_VAL,POPULATION_SHAP_VAL
0,-1.316280,-2.046322,-2.007526,-0.034098,0.352032,0.307791,-2.073674,6.801652,-12.832669,-6.261691,1.912319,3.930407,-1.424739,-0.321290,-2.015752,15.557734
1,-2.250615,-5.679394,13.626263,1.218979,-0.690839,1.963175,-0.571189,9.120384,-17.305241,-4.655366,-0.283124,-0.912725,-0.465578,-2.574101,-1.709512,31.443420
2,4.293972,0.801322,-25.479273,2.690810,-2.580008,9.148360,1.392307,-9.743123,5.681876,-3.047925,0.202694,1.195912,0.246232,-0.794824,1.486046,29.993429
3,2.357388,-2.525553,-8.203782,1.365706,-0.981811,0.349225,-0.218600,9.609642,-17.985758,-5.564096,5.127561,4.344897,3.679505,-0.338709,3.410195,31.994444
4,-0.742292,-0.185635,-2.129418,-2.686177,1.308070,-8.451877,2.057759,3.385180,-5.980525,-3.098035,-2.428781,3.645219,2.044959,0.716674,2.521634,-6.746540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72903,-1.989110,-2.816174,-3.116372,-0.068332,-0.749464,-2.926788,-1.002457,-3.570291,-6.676291,-3.078245,-2.868962,-3.739931,5.418073,-0.383719,2.178583,18.725983
72904,1.741008,-4.022261,10.514369,-0.641506,2.795826,-0.277708,-0.808833,17.968739,-15.587749,-6.822184,3.395998,5.760224,-0.152776,0.054388,-3.296051,41.047558
72905,1.606852,-5.784650,-0.569128,0.742261,-2.323673,6.654408,1.096931,12.808215,-18.644543,-4.588741,3.198109,3.656629,-0.413819,0.478324,-2.428452,37.341507
72906,1.026393,0.669005,-20.038170,2.266943,-4.007723,5.430455,-5.293857,-10.582945,1.051467,-1.915721,-13.329426,0.244185,0.117338,-0.336419,-3.957954,25.791721


In [68]:
combined_df = pd.concat([all_data['FIPS'], shap_df], axis=1)
combined_df['EVICTIONS_PRED'] = preds
combined_df['EVICTIONS'] = all_data['EVICTIONS']

In [69]:
combined_df

,FIPS,EP_UNEMP_SHAP_VAL,EP_PCI_SHAP_VAL,EP_NOVEH_SHAP_VAL,EP_MOBILE_SHAP_VAL,EP_CROWD_SHAP_VAL,EP_AGE65_SHAP_VAL,EP_AGE17_SHAP_VAL,AFAM_SHAP_VAL,WHITE_SHAP_VAL,HISPANIC_SHAP_VAL,WFIR_EALT_SHAP_VAL,HRCN_EALT_SHAP_VAL,CFLD_EALT_SHAP_VAL,RFLD_EALT_SHAP_VAL,HWAV_EALT_SHAP_VAL,POPULATION_SHAP_VAL,EVICTIONS_PRED,EVICTIONS
0,39061025102GID,-1.316280,-2.046322,-2.007526,-0.034098,0.352032,0.307791,-2.073674,6.801652,-12.832669,-6.261691,1.912319,3.930407,-1.424739,-0.321290,-2.015752,15.557734,20.089855,5.0
1,42101027000GID,-2.250615,-5.679394,13.626263,1.218979,-0.690839,1.963175,-0.571189,9.120384,-17.305241,-4.655366,-0.283124,-0.912725,-0.465578,-2.574101,-1.709512,31.443420,41.836483,NaN
2,06031000300GID,4.293972,0.801322,-25.479273,2.690810,-2.580008,9.148360,1.392307,-9.743123,5.681876,-3.047925,0.202694,1.195912,0.246232,-0.794824,1.486046,29.993429,37.049770,NaN
3,24025301402GID,2.357388,-2.525553,-8.203782,1.365706,-0.981811,0.349225,-0.218600,9.609642,-17.985758,-5.564096,5.127561,4.344897,3.679505,-0.338709,3.410195,31.994444,47.982193,NaN
4,51097950500GID,-0.742292,-0.185635,-2.129418,-2.686177,1.308070,-8.451877,2.057759,3.385180,-5.980525,-3.098035,-2.428781,3.645219,2.044959,0.716674,2.521634,-6.746540,4.792177,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72903,34025809902GID,-1.989110,-2.816174,-3.116372,-0.068332,-0.749464,-2.926788,-1.002457,-3.570291,-6.676291,-3.078245,-2.868962,-3.739931,5.418073,-0.383719,2.178583,18.725983,14.898465,NaN
72904,09009141100GID,1.741008,-4.022261,10.514369,-0.641506,2.795826,-0.277708,-0.808833,17.968739,-15.587749,-6.822184,3.395998,5.760224,-0.152776,0.054388,-3.296051,41.047558,73.231010,NaN
72905,29095009200GID,1.606852,-5.784650,-0.569128,0.742261,-2.323673,6.654408,1.096931,12.808215,-18.644543,-4.588741,3.198109,3.656629,-0.413819,0.478324,-2.428452,37.341507,54.392185,NaN
72906,06029005700GID,1.026393,0.669005,-20.038170,2.266943,-4.007723,5.430455,-5.293857,-10.582945,1.051467,-1.915721,-13.329426,0.244185,0.117338,-0.336419,-3.957954,25.791721,-1.302742,NaN


In [70]:
combined_df.to_csv('../data/predictions_shap.csv')